# 9. Triggers in RDBMS (MySQL)

A **trigger** is a set of instructions that are automatically executed (or "triggered") in response to a specific event on a particular table in a database. These events can be `INSERT`, `UPDATE`, or `DELETE` operations.

#### Key Concepts:

1. **Trigger Timing**: Determines when the trigger's action should be executed in relation to the triggering event. The two options are:
   - `BEFORE`: The trigger is executed before the event.
   - `AFTER`: The trigger is executed after the event.

2. **Trigger Event**: The type of SQL operation that activates the trigger. The three options are:
   - `INSERT`
   - `UPDATE`
   - `DELETE`

3. **Trigger Body**: The SQL statements that are executed when the trigger is activated.

4. **OLD and NEW Keywords**: These are used to refer to the values of the affected rows.
   - `OLD`: Refers to the row values before the triggering event.
   - `NEW`: Refers to the row values after the triggering event.

### Example

Let's create a simple example to illustrate how to use triggers in MySQL. We'll use two tables: `employees` and `audit_log`.

- **employees**: Stores information about employees.
- **audit_log**: Keeps a log of changes made to the `employees` table.

#### Step-by-Step Example

1. **Create the Tables**

```sql
CREATE TABLE employees (
    employee_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100),
    position VARCHAR(100),
    salary DECIMAL(10, 2)
);

CREATE TABLE audit_log (
    log_id INT AUTO_INCREMENT PRIMARY KEY,
    employee_id INT,
    action VARCHAR(50),
    action_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    old_salary DECIMAL(10, 2),
    new_salary DECIMAL(10, 2)
);
```

2. **Create the Trigger**

The following trigger will log any changes to an employee's salary in the `audit_log` table.

```sql
DELIMITER //

CREATE TRIGGER before_employee_update
BEFORE UPDATE ON employees
FOR EACH ROW
BEGIN
    IF NEW.salary != OLD.salary THEN
        INSERT INTO audit_log (employee_id, action, old_salary, new_salary)
        VALUES (OLD.employee_id, 'Salary Update', OLD.salary, NEW.salary);
    END IF;
END;

//

DELIMITER ;
```

In this example:

- The trigger is named `before_employee_update`.
- It is a `BEFORE UPDATE` trigger, meaning it will activate before an `UPDATE` operation on the `employees` table.
- The trigger checks if the `salary` field is being changed (`IF NEW.salary != OLD.salary`).
- If the salary is changed, it inserts a record into the `audit_log` table, capturing the employee's ID, action description, old salary, and new salary.

3. **Test the Trigger**

Insert a sample record into the `employees` table:

```sql
INSERT INTO employees (name, position, salary) VALUES ('John Doe', 'Manager', 60000);
```

Update the employee's salary:

```sql
UPDATE employees SET salary = 65000 WHERE employee_id = 1;
```

Check the `audit_log` table to see the log entry:

```sql
SELECT * FROM audit_log;
```

You should see a record indicating the salary change for the employee.

### Summary

Triggers are powerful tools in RDBMS like MySQL that allow you to automatically execute actions in response to certain events. This example demonstrates how to use triggers to maintain an audit log of changes to an employee's salary. The principles can be applied to other scenarios as needed.

### Example Inventory Management System
```sql
-- Create the Triggers database and use it
CREATE DATABASE Triggers;
USE Triggers;

-- Drop tables and database if they already exist (uncomment if needed)
-- DROP TABLE IF EXISTS Products;
-- DROP TABLE IF EXISTS inventory_log;
-- DROP DATABASE IF EXISTS TRIGGERS;

-- Create Products table
CREATE TABLE Products(
    product_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100),
    stock INT
);

-- Create inventory_log table
CREATE TABLE inventory_log (
    log_id INT AUTO_INCREMENT PRIMARY KEY,
    product_id INT,
    change_type VARCHAR(50),
    change_amount INT,
    change_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Insert initial data into Products table
INSERT INTO Products (product_id, name, stock) VALUES (1,'Laptop',10);
INSERT INTO Products (product_id, name, stock) VALUES (2,'Monitor',8);

-- Select all data from Products table
SELECT * FROM Products;

-- Create AFTER UPDATE trigger to log changes in stock
DELIMITER //

CREATE TRIGGER after_stock_update 
AFTER UPDATE ON Products
FOR EACH ROW
BEGIN
    DECLARE change_amount INT;

    -- Calculate the change in stock
    SET change_amount = NEW.stock - OLD.stock;

    -- Insert into inventory_log table
    INSERT INTO inventory_log (product_id, change_type, change_amount)
    VALUES (NEW.product_id, 'Stock Change', change_amount);
END;
//

DELIMITER ;

-- Update the stock of Laptop from 10 to 15
UPDATE Products SET stock = 15 WHERE product_id = 1;

-- Update the stock of Monitor from 5 to 3
UPDATE Products SET stock = 3 WHERE product_id = 2;

-- Select all data from Products and inventory_log tables
SELECT * FROM Products;
SELECT * FROM inventory_log;

-- Create BEFORE UPDATE trigger to log changes in stock
DELIMITER //

CREATE TRIGGER before_stock_update 
BEFORE UPDATE ON Products
FOR EACH ROW
BEGIN
    DECLARE change_amount INT;

    -- Calculate the change in stock
    SET change_amount = NEW.stock - OLD.stock;

    -- Insert into inventory_log table
    INSERT INTO inventory_log (product_id, change_type, change_amount)
    VALUES (NEW.product_id, 'Stock Change', change_amount);
END;
//

DELIMITER ;
```

### Explanation
1. **Database Creation**:
   - `CREATE DATABASE Triggers;` creates a new database named `Triggers`.
   - `USE Triggers;` switches to the `Triggers` database for subsequent operations.

2. **Table Creation**:
   - `CREATE TABLE Products(...);` creates the `Products` table with columns for `product_id`, `name`, and `stock`.
   - `CREATE TABLE inventory_log(...);` creates the `inventory_log` table with columns for `log_id`, `product_id`, `change_type`, `change_amount`, and `change_time`.

3. **Data Insertion**:
   - `INSERT INTO Products...;` inserts initial product data into the `Products` table.

4. **Trigger Creation**:
   - The `AFTER UPDATE` trigger named `after_stock_update` logs changes in stock to the `inventory_log` table after any update on the `Products` table.
   - The `BEFORE UPDATE` trigger named `before_stock_update` logs changes in stock to the `inventory_log` table before any update on the `Products` table.

5. **Data Update**:
   - `UPDATE Products SET stock = 15 WHERE product_id = 1;` updates the stock of the product with `product_id` 1 (Laptop) to 15.
   - `UPDATE Products SET stock = 3 WHERE product_id = 2;` updates the stock of the product with `product_id` 2 (Monitor) to 3.

6. **Data Selection**:
   - `SELECT * FROM Products;` and `SELECT * FROM inventory_log;` retrieve and display all records from the `Products` and `inventory_log` tables respectively.

### Note
- The `DELIMITER //` and `DELIMITER ;` commands are used to change the statement delimiter so that the entire trigger creation statement can be treated as a single statement.
- The `DECLARE` and `SET` statements within the triggers are used to calculate the change in stock.
- The triggers log the changes in the `inventory_log` table with appropriate `change_type` and `change_amount`.

#### Prepared By,
Ahamed Basith